# Static - Interview Question Creation Based On Resume
> In this agent we will be implementing an AI which will look into resume and generate interview questions to evaluate candidate.

####  Step 1 - Ensure all environment variables are available and initialize LLM(s).

In [111]:
# Load Environment Variables

import os

from dotenv import load_dotenv
load_dotenv()

print('LOADED OPENAI_API_KEY :=',os.environ['OPENAI_API_KEY'] == os.getenv('OPENAI_API_KEY'))
print('LOADED LANGSMITH_API_KEY :=',os.environ['LANGSMITH_API_KEY'] == os.getenv('LANGSMITH_API_KEY'))
print('LOADED LANGSMITH_TRACING :=',os.environ['LANGSMITH_TRACING'] == os.getenv('LANGSMITH_TRACING'))

LOADED OPENAI_API_KEY := True
LOADED LANGSMITH_API_KEY := True
LOADED LANGSMITH_TRACING := True


In [112]:
# Create a LLM instance
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo")

####  Step 2 - Creating State in order to go ahead with workflow.

In [113]:
from typing import List
from typing_extensions import TypedDict, NotRequired, Literal, Annotated

import operator

# --- Leaf / child models ---
class ResumeAdditionalAward(TypedDict, total=False):
    title: NotRequired[str]

class ResumeCertification(TypedDict, total=False):
    certificate_name: NotRequired[str]
    certificate_link: NotRequired[str]
    issued_by: NotRequired[str]

class ResumeEducation(TypedDict, total=False):
    institution: NotRequired[str]
    location: NotRequired[str]
    degree_type: NotRequired[str]
    field_of_study: NotRequired[str]
    start_month_year: NotRequired[str]
    end_month_year: NotRequired[str]
    score_metric: NotRequired[str]
    score: NotRequired[str]

class ResumeExperience(TypedDict, total=False):
    employer: NotRequired[str]
    Job_title: NotRequired[str]
    location: NotRequired[str]
    start_month_year: NotRequired[str]
    end_month_year: NotRequired[str]

class ResumePersonalInfo(TypedDict, total=False):
    first_name: NotRequired[str]
    last_name: NotRequired[str]
    email: NotRequired[str]
    phone: NotRequired[str]
    address: NotRequired[str]
    dob: NotRequired[str]
    job_title: NotRequired[str]
    git_link: NotRequired[str]
    linkedin_profile: NotRequired[str]
    portfolio_link: NotRequired[str]

class ResumeProject(TypedDict, total=False):
    project_name: NotRequired[str]
    technologies_used: NotRequired[str]
    project_link: NotRequired[str]
    description: NotRequired[str]

class ResumeSkillSet(TypedDict, total=False):
    programming_languages: NotRequired[List[str]]
    libraries_and_frameworks: NotRequired[List[str]]
    tools_and_platforms: NotRequired[List[str]]
    databases: NotRequired[List[str]]
    concepts: NotRequired[List[str]]

class InterviewQuestion(TypedDict, total=False):
    id: str
    question_text: str
    difficulty_level: NotRequired[str]
    based_on: Literal['education', 'experience', 'skills', 'additional_awards', 'certifications', 'projects', 'personal_info', 'greet_and_small_talk']

# --- Root Resume model ---
class Resume(TypedDict, total=False):
    id: str
    profile_name: NotRequired[str]
    InterviewQuestion: Annotated[List[InterviewQuestion], operator.add]
    
    ResumeAdditionalAward: NotRequired[List[ResumeAdditionalAward]]
    ResumeCertification: NotRequired[List[ResumeCertification]]
    ResumeEducation: NotRequired[List[ResumeEducation]]
    ResumeExperience: NotRequired[List[ResumeExperience]]
    ResumePersonalInfo: ResumePersonalInfo
    ResumeProject: NotRequired[List[ResumeProject]]
    ResumeSkillSet: NotRequired[List[ResumeSkillSet]]

In [114]:
# Sample Data
sample_resume: Resume = {
    "id": "a1b2c3d4-e5f6-7890-abcd-ef1234567890",
    "profile_name": "Backend Developer Resume",

    "ResumeAdditionalAward": [
        {"title": "Best Software Engineer Award 2023"},
        {"title": "Top Contributor - Open Source"}
    ],

    "ResumeCertification": [
        {
            "certificate_name": "AWS Certified Solutions Architect",
            "certificate_link": "https://aws.amazon.com/certification/",
            "issued_by": "Amazon Web Services"
        },
        {
            "certificate_name": "Google Cloud Professional Developer",
            "certificate_link": "https://cloud.google.com/certification",
            "issued_by": "Google Cloud"
        }
    ],

    "ResumeEducation": [
        {
            "institution": "Indian Institute of Technology Bombay",
            "location": "Mumbai, India",
            "degree_type": "B.Tech",
            "field_of_study": "Computer Science",
            "start_month_year": "Aug 2017",
            "end_month_year": "May 2021",
            "score_metric": "CGPA",
            "score": "8.7"
        }
    ],

    "ResumeExperience": [
        {
            "employer": "Coditas",
            "Job_title": "Software Engineer",
            "location": "Pune, India",
            "start_month_year": "Jun 2021",
            "end_month_year": "Aug 2023"
        },
        {
            "employer": "Jash Datasciences",
            "Job_title": "Backend Developer",
            "location": "Remote",
            "start_month_year": "Sep 2023",
            "end_month_year": "Present"
        }
    ],

    "ResumePersonalInfo": {
            "first_name": "Suyash",
            "last_name": "Lawand",
            "email": "suyash@example.com",
            "phone": "+91-9876543210",
            "address": "Pune, Maharashtra, India",
            "dob": "1999-04-15",
            "job_title": "Backend Developer",
            "git_link": "https://github.com/suyashlawand",
            "linkedin_profile": "https://linkedin.com/in/suyashlawand",
            "portfolio_link": "https://suyashlawand.dev"
        },

    "ResumeProject": [
        {
            "project_name": "Menyokard",
            "technologies_used": "NestJS, PostgreSQL, Docker",
            "project_link": "https://menyokard.com",
            "description": "Real-time café management platform with QR-based ordering and billing."
        },
        {
            "project_name": "AI Resume Evaluator",
            "technologies_used": "Next.js, LangChain, Python",
            "project_link": "https://aireview.ai",
            "description": "An AI-driven resume analyzer with mock interview functionality."
        }
    ],

    "ResumeSkillSet": [
        {
            "programming_languages": ["Python", "Java", "TypeScript"],
            "libraries_and_frameworks": ["NestJS", "React", "Spring Boot"],
            "tools_and_platforms": ["Docker", "Kubernetes", "GitHub Actions"],
            "databases": ["PostgreSQL", "MySQL", "MongoDB"],
            "concepts": ["Microservices", "REST APIs", "Distributed Systems"]
        }
    ],

    "InterviewQuestion": []
}

####  Step 3 - Creating nodes in order to go ahead with workflow.

In [115]:
from uuid import uuid4
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import json

def greet_and_small_talk(resume: Resume) -> Resume:
    # Pull first personal-info block safely (schema stores it as a list)
    pi = resume.get("ResumePersonalInfo")


    first_name = (pi.get("first_name") or "").strip()
    last_name = (pi.get("last_name") or "").strip()
    job_title = (pi.get("job_title") or resume.get("profile_name") or "the role").strip()
    profile_name = (resume.get("profile_name") or "").strip()
    current_location = (pi.get("address") or "").strip()

    greeting_prompt = PromptTemplate(
        input_variables=["first_name", "last_name", "job_title", "profile_name", "current_location"],
        template="""
You are a friendly AI interviewer.

Your only task is to greet the candidate warmly before starting the interview.
Use these details to personalize:

- First Name: {first_name}
- Last Name: {last_name}
- Job Title: {job_title}
- Resume Profile Name: {profile_name}
- Current Location: {current_location}

Guidelines:
- Keep it short (2–3 sentences)
- Professional but friendly
- Acknowledge location if given
- Do NOT ask interview questions

Return a json list of strings with two questions doing general talk.
""".strip()
    )

    chain = greeting_prompt | llm | StrOutputParser()
    result = chain.invoke({
        "first_name": first_name,
        "last_name": last_name,
        "job_title": job_title,
        "profile_name": profile_name,
        "current_location": current_location,
    })

    # Extract text from common LC return types
    greeting_text = result

    question: InterviewQuestion = {
        "id": str(uuid4()),
        "question_text": greeting_text,
        "difficulty_level": "easy",
        "based_on": "greet_and_small_talk",
    }

    resume.setdefault("InterviewQuestion", []).append(question)
    return resume


In [116]:
greet_and_small_talk(sample_resume)

{'id': 'a1b2c3d4-e5f6-7890-abcd-ef1234567890',
 'profile_name': 'Backend Developer Resume',
 'ResumeAdditionalAward': [{'title': 'Best Software Engineer Award 2023'},
  {'title': 'Top Contributor - Open Source'}],
 'ResumeCertification': [{'certificate_name': 'AWS Certified Solutions Architect',
   'certificate_link': 'https://aws.amazon.com/certification/',
   'issued_by': 'Amazon Web Services'},
  {'certificate_name': 'Google Cloud Professional Developer',
   'certificate_link': 'https://cloud.google.com/certification',
   'issued_by': 'Google Cloud'}],
 'ResumeEducation': [{'institution': 'Indian Institute of Technology Bombay',
   'location': 'Mumbai, India',
   'degree_type': 'B.Tech',
   'field_of_study': 'Computer Science',
   'start_month_year': 'Aug 2017',
   'end_month_year': 'May 2021',
   'score_metric': 'CGPA',
   'score': '8.7'}],
 'ResumeExperience': [{'employer': 'Coditas',
   'Job_title': 'Software Engineer',
   'location': 'Pune, India',
   'start_month_year': 'Jun 20